# Time-Domain Simulation

Time-domain simulation (TDS) solves the differential-algebraic equations (DAEs) that govern power system dynamics. This type of analysis is essential for studying transient stability, which determines whether a power system can maintain synchronism following disturbances such as faults, line trips, or generator outages.

ANDES uses the implicit trapezoidal method for numerical integration, which provides a good balance between accuracy and computational efficiency. The method is A-stable, meaning it handles stiff systems well without requiring excessively small time steps.

This tutorial covers running time-domain simulations, adding various types of disturbances, configuring simulation parameters, accessing results, and troubleshooting common issues.

In [ ]:
# Reduce logging verbosity for PDF builds
import os
if os.environ.get('SPHINX_BUILD_PDF'):
    import andes
    _orig_config_logger = andes.config_logger
    def _quiet_logger(stream_level=20, *args, **kwargs):
        stream_level = max(stream_level, 30)
        return _orig_config_logger(stream_level, *args, **kwargs)
    andes.config_logger = _quiet_logger

## Setup

In [1]:
%matplotlib inline

import andes
andes.config_logger(stream_level=20)

## Basic Simulation Workflow

A time-domain simulation always begins with a converged power flow solution, which provides the initial steady-state operating point. From this equilibrium, the DAE solver advances time while tracking how system variables evolve in response to any disturbances.

The basic workflow consists of three steps: load the case, run power flow, then run TDS. The simulation end time is controlled by `ss.TDS.config.tf`.

In [2]:
ss = andes.load(andes.get_case('kundur/kundur_full.xlsx'))
ss.PFlow.run()

ss.TDS.config.tf = 10  # Simulate for 10 seconds
ss.TDS.run()

Working directory: "/Users/hcui7/repos/andes/docs_new/source/tutorials"


> Loaded config from file "/Users/hcui7/.andes/andes.rc"


> Loaded generated Python code in "/Users/hcui7/.andes/pycode".


Parsing input file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"...


Input file parsed in 0.3178 seconds.


...

<Toggle 1>: Line.Line_8 status changed to 0 at t=2.0 sec.


Simulation to t=10.00 sec completed in 0.3990 seconds.


Outputs to "kundur_full_out.lst" and "kundur_full_out.npz".


Outputs written in 0.0094 seconds.


True

The Kundur test case includes a pre-defined line trip at t=2 seconds, which triggers electromechanical oscillations between the two areas. The progress bar shows simulation advancement, and the final message confirms successful completion.

### Command Line Alternative

Time-domain simulation can be run from the command line using the `-r tds` flag. The `--tf` option sets the simulation end time.

```bash
andes run kundur_full.xlsx -r tds --tf 20
```

## Adding Disturbances

Disturbances are essential for transient stability analysis because they perturb the system from its equilibrium and allow you to observe the dynamic response. ANDES provides three main disturbance devices:

| Device | Purpose |
|--------|---------|
| `Fault` | Three-phase-to-ground bus fault |
| `Toggle` | Connect or disconnect any device |
| `Alter` | Change parameter values during simulation |

To add disturbances, you must load the case with `setup=False`, add the disturbance devices, then call `setup()` before running. This sequence is necessary because the system structure is finalized during setup.

:::{seealso}
For details on how timed events are implemented internally using `TimerParam` callbacks, see {doc}`../modeling/concepts/system-architecture`.
:::

### Three-Phase Faults

The `Fault` model applies a three-phase-to-ground fault on a bus. This is the most severe type of fault and is commonly used for stability screening. The fault is characterized by its location (bus), application time (`tf`), and clearing time (`tc`).

In [3]:
ss_fault = andes.load(andes.get_case('kundur/kundur_full.xlsx'), setup=False)

# Disable the existing Toggle to isolate the fault effect
ss_fault.Toggle.alter('u', 1, 0)

# Add fault on bus 5: apply at t=1.0s, clear at t=1.1s
ss_fault.add('Fault', {
    'bus': 5,
    'tf': 1.0,
    'tc': 1.1
})

ss_fault.setup()
ss_fault.PFlow.run()
ss_fault.TDS.config.tf = 5
ss_fault.TDS.run()

Working directory: "/Users/hcui7/repos/andes/docs_new/source/tutorials"


> Loaded config from file "/Users/hcui7/.andes/andes.rc"


> Reloaded generated Python code of module "pycode".


Parsing input file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"...


Input file parsed in 0.0297 seconds.


...

Simulation to t=5.00 sec completed in 0.1787 seconds.


Outputs to "kundur_full_out.lst" and "kundur_full_out.npz".


Outputs written in 0.0041 seconds.


<Fault Fault_2>: Applying fault on Bus (idx=5) at t=1.0 sec.
<Fault Fault_2>: Clearing fault on Bus (idx=5) at t=1.1 sec.


True

The fault parameters control its severity and duration:

| Parameter | Description | Default |
|-----------|-------------|--------|
| `bus` | Bus index for fault location | (required) |
| `tf` | Fault application time [s] | (required) |
| `tc` | Fault clearing time [s] | (required) |
| `rf` | Fault resistance [pu] | 0 |
| `xf` | Fault reactance [pu] | 1e-6 |

For a permanent fault that is never cleared, set `tc` to a time beyond the simulation end.

In [4]:
ss_fault.TDS.plt.plot(ss_fault.GENROU.omega, ylabel='Generator Speed [pu]')

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


(<Figure size 640x480 with 1 Axes>,
 <Axes: xlabel='Time [s]', ylabel='Generator Speed [pu]'>)

### Line and Device Trips (Toggle)

The `Toggle` model changes the connectivity status of any device at a specified time. This is useful for simulating line trips, generator trips, load shedding, or breaker operations. Unlike faults which modify the network admittance, toggles completely remove or reconnect devices.

The following example demonstrates a line trip followed by reclosure.

In [5]:
ss_toggle = andes.load(andes.get_case('kundur/kundur_full.xlsx'), setup=False)

# Disable existing Toggle
ss_toggle.Toggle.alter('u', 1, 0)

# Trip line at t=1.0s, reclose at t=1.5s
ss_toggle.add('Toggle', {'model': 'Line', 'dev': 'Line_5', 't': 1.0})
ss_toggle.add('Toggle', {'model': 'Line', 'dev': 'Line_5', 't': 1.5})

ss_toggle.setup()
ss_toggle.PFlow.run()
ss_toggle.TDS.config.tf = 5
ss_toggle.TDS.run()

Working directory: "/Users/hcui7/repos/andes/docs_new/source/tutorials"


> Loaded config from file "/Users/hcui7/.andes/andes.rc"


> Reloaded generated Python code of module "pycode".


Parsing input file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"...


Input file parsed in 0.0192 seconds.


...

Simulation to t=5.00 sec completed in 0.1192 seconds.


Outputs to "kundur_full_out.lst" and "kundur_full_out.npz".


Outputs written in 0.0084 seconds.


<Toggle Toggle_2>: Line.Line_5 status changed to 0 at t=1.0 sec.
<Toggle Toggle_3>: Line.Line_5 status changed to 1 at t=1.5 sec.


True

In [6]:
ss_toggle.TDS.plt.plot(ss_toggle.GENROU.omega, ylabel='Generator Speed [pu]')

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


(<Figure size 640x480 with 1 Axes>,
 <Axes: xlabel='Time [s]', ylabel='Generator Speed [pu]'>)

Toggle can operate on any model type. Common applications include:

- **Generator trip**: `{'model': 'GENROU', 'dev': 'GENROU_2', 't': 1.0}`
- **Load shedding**: `{'model': 'PQ', 'dev': 'PQ_1', 't': 2.0}`
- **Transformer trip**: `{'model': 'Line', 'dev': 'Line_10', 't': 0.5}`

### Parameter Changes (Alter)

The `Alter` model changes parameter or service values at specified times during simulation. This is useful for studying the effect of setpoint changes, such as adjusting governor references or exciter voltage setpoints. The alteration is specified by the source field (`src`), the method (`=`, `+`, `-`, `*`, `/`), and the amount to apply.

In [7]:
ss_alter = andes.load(andes.get_case('kundur/kundur_full.xlsx'), setup=False)

# Disable existing Toggle
ss_alter.Toggle.alter('u', 1, 0)

# Step change in governor setpoint at t=1.0s
ss_alter.add('Alter', {
    'model': 'TGOV1',
    'dev': 1,
    'src': 'pref0',
    't': 1.0,
    'method': '=',
    'amount': 0.8  # New setpoint value
})

ss_alter.setup()
ss_alter.PFlow.run()
ss_alter.TDS.config.tf = 10
ss_alter.TDS.run()

Working directory: "/Users/hcui7/repos/andes/docs_new/source/tutorials"


> Loaded config from file "/Users/hcui7/.andes/andes.rc"


> Reloaded generated Python code of module "pycode".


Parsing input file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"...


Input file parsed in 0.0194 seconds.


...

<Alter Alter_2>: set TGOV1.1.pref0.v=0.8 at t=1. Previous value was 7.26803.


Simulation to t=10.00 sec completed in 0.2959 seconds.


Outputs to "kundur_full_out.lst" and "kundur_full_out.npz".


Outputs written in 0.0088 seconds.


True

In [8]:
ss_alter.TDS.plt.plot(ss_alter.GENROU.omega, ylabel='Generator Speed [pu]')

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


(<Figure size 640x480 with 1 Axes>,
 <Axes: xlabel='Time [s]', ylabel='Generator Speed [pu]'>)

## Configuration Options

The TDS routine provides several configuration options that control simulation behavior. These can be accessed and modified through `ss.TDS.config`.

In [9]:
ss.TDS.config

OrderedDict([('sparselib', 'klu'),
             ('linsolve', 0),
             ('method', 'trapezoid'),
             ('tol', 0.0001),
             ('t0', 0.0),
             ('tf', 20.0),
             ('fixt', 1),
             ('shrinkt', 1),
             ('honest', 0),
             ('tstep', 0.03333333333333333),
             ('max_iter', 15),
             ('refresh_event', 0),
             ('test_init', 1),
             ('check_conn', 1),
             ('criteria', 1),
             ('ddelta_limit', 180),
             ('g_scale', 1),
             ('reset_tiny', 1),
             ('qrt', 0),
             ('kqrt', 1.0),
             ('store_z', 0),
             ('store_f', 0),
             ('store_h', 0),
             ('store_i', 0),
             ('limit_store', 0),
             ('max_store', 900),
             ('save_every', 1),
             ('save_mode', 'auto'),
             ('no_tqdm', 0),
             ('chatter_iter', 4)])

The most commonly adjusted options are:

| Option | Default | Description |
|--------|---------|-------------|
| `tf` | 20 | Simulation end time [seconds] |
| `tstep` | 1/30 | Integration time step [seconds] |
| `max_iter` | 15 | Maximum Newton iterations per step |
| `tol` | 1e-6 | Convergence tolerance |
| `fixt` | 1 | Use fixed (1) or variable (0) step size |

For simulations with fast dynamics or discontinuities, you may need to reduce the time step to maintain accuracy and stability.

## Accessing Results

Time series data from the simulation is stored in the DAE time series object `ss.dae.ts`. This object contains arrays for time, state variables, and algebraic variables at each saved time point.

### Time Series Structure

In [10]:
# Time vector
t = ss.dae.ts.t
print(f"Time points: {len(t)}, from {t[0]:.2f} to {t[-1]:.2f} seconds")

# State variables: shape (n_steps, n_states)
print(f"State variable array shape: {ss.dae.ts.x.shape}")

# Algebraic variables: shape (n_steps, n_algebs)
print(f"Algebraic variable array shape: {ss.dae.ts.y.shape}")

Time points: 303, from 0.00 to 10.00 seconds
State variable array shape: (303, 52)
Algebraic variable array shape: (303, 149)


### Extracting Specific Variables

To extract a specific variable's time series, you need to know its address in the state or algebraic vector. Each variable object has an `a` attribute that returns its address(es). For state variables, use `ss.dae.ts.x`; for algebraic variables, use `ss.dae.ts.y`.

In [11]:
# Get generator speed addresses
omega_addr = ss.GENROU.omega.a
print(f"Generator speed addresses: {omega_addr}")

# Extract time series for all generators
omega_ts = ss.dae.ts.x[:, omega_addr]
print(f"Speed time series shape: {omega_ts.shape}")

Generator speed addresses: [4 5 6 7]
Speed time series shape: (303, 4)


In [12]:
# Bus voltage is an algebraic variable
v_addr = ss.Bus.v.a
voltage_ts = ss.dae.ts.y[:, v_addr]
print(f"Voltage time series shape: {voltage_ts.shape}")

Voltage time series shape: (303, 10)


## Plotting Results

ANDES provides a built-in plotter accessible through `ss.TDS.plt` that simplifies visualization of simulation results. The `plot()` method accepts variable objects and automatically handles address lookup and data extraction.

In [13]:
# Plot all generator speeds
ss.TDS.plt.plot(ss.GENROU.omega)

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


(<Figure size 640x480 with 1 Axes>, <Axes: xlabel='Time [s]'>)

In [14]:
# Plot specific generators using address indices
ss.TDS.plt.plot(ss.GENROU.omega, a=[0, 2], ylabel='Speed [pu]')

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


(<Figure size 640x480 with 1 Axes>,
 <Axes: xlabel='Time [s]', ylabel='Speed [pu]'>)

### Exporting Results

Results can be exported to CSV format for further analysis in other tools.

In [15]:
ss.TDS.plt.export_csv('kundur_results.csv')

CSV data saved to "kundur_results.csv".


'kundur_results.csv'

## Multi-Stage Simulation

For complex scenarios, you can run simulation in multiple stages with parameter changes between stages. After completing one stage, simply update the end time and call `run()` again. ANDES will continue from where it left off, preserving all state variables.

In [16]:
ss_multi = andes.load(andes.get_case('kundur/kundur_full.xlsx'), setup=False)
ss_multi.Toggle.alter('u', 1, 0)  # Disable existing toggle
ss_multi.setup()
ss_multi.PFlow.run()

# Stage 1: Run to t=2s
ss_multi.TDS.config.tf = 2.0
ss_multi.TDS.run()

# Apply setpoint change between stages
ss_multi.TGOV1.paux0.v[0] = 0.05

# Stage 2: Continue to t=10s
ss_multi.TDS.config.tf = 10.0
ss_multi.TDS.run()

Working directory: "/Users/hcui7/repos/andes/docs_new/source/tutorials"


> Loaded config from file "/Users/hcui7/.andes/andes.rc"


> Reloaded generated Python code of module "pycode".


Parsing input file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"...


Input file parsed in 0.0183 seconds.


...

  0%|          | 0/100 [00:00<?, ?%/s]

Simulation to t=10.00 sec completed in 0.2110 seconds.


Outputs to "kundur_full_out.lst" and "kundur_full_out.npz".


Outputs written in 0.0071 seconds.


True

In [17]:
# Plot shows both stages seamlessly
ss_multi.TDS.plt.plot(ss_multi.GENROU.omega)

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


(<Figure size 640x480 with 1 Axes>, <Axes: xlabel='Time [s]'>)

## Output Files

TDS automatically generates output files containing simulation results:

| File | Description |
|------|-------------|
| `*_out.lst` | Variable listing with indices and names |
| `*_out.npz` | Compressed NumPy archive with time series data |

These files can be loaded later for post-processing without re-running the simulation.

In [18]:
import numpy as np

# Load saved results
data = np.load(ss.files.npz)
print("Available arrays:", list(data.keys()))

Available arrays: ['data']


## Troubleshooting

### Simulation Fails to Converge

If the simulation fails during integration, the Newton iteration at a particular time step did not converge within the allowed iterations. This usually indicates:

1. **Step size too large**: Reduce `tstep` to capture faster dynamics
2. **Severe disturbance**: The system may be transiently unstable
3. **Model limitations**: Some operating points may exceed model validity ranges

In [19]:
# Typical remedies for convergence issues
ss.TDS.config.tstep = 0.001  # Smaller step size
ss.TDS.config.max_iter = 30  # More iterations allowed

### Checking Initialization

Before running a full simulation, you can initialize TDS without running to check for any initialization errors. This helps identify problems with initial conditions before committing to a long simulation.

In [20]:
ss_check = andes.load(andes.get_case('kundur/kundur_full.xlsx'))
ss_check.PFlow.run()
ss_check.TDS.init()  # Initialize only, don't run

Working directory: "/Users/hcui7/repos/andes/docs_new/source/tutorials"


> Loaded config from file "/Users/hcui7/.andes/andes.rc"


> Reloaded generated Python code of module "pycode".


Parsing input file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"...


Input file parsed in 0.0190 seconds.


...

Converged in 5 iterations in 0.0024 seconds.


Report saved to "kundur_full_out.txt" in 0.0006 seconds.


Initialization for dynamics completed in 0.0128 seconds.


Initialization was successful.


array([ 1.41994832,  1.12395566,  0.93892052,  1.2113756 ,  1.        ,
        1.        ,  1.        ,  1.        ,  0.86626509,  0.94860469,
        0.95125534,  0.86870301,  0.50808219,  0.45912202,  0.4577068 ,
        0.50085419,  0.70142378,  0.77725187,  0.77932429,  0.71147986,
        0.72456938,  0.65474793,  0.6527297 ,  0.71426162,  7.26802921,
        7.        ,  7.        ,  7.        ,  7.26802921,  7.        ,
        7.        ,  7.        ,  1.89652323,  2.0195598 ,  2.02582444,
        1.85134766,  1.        ,  1.        ,  1.        ,  1.        ,
        0.09482616,  0.10097799,  0.10129122,  0.09256738,  1.89652323,
        2.0195598 ,  2.02582444,  1.85134766,  1.89652323,  2.0195598 ,
        2.02582444,  1.85134766,  0.57025492,  0.3779617 ,  0.19577145,
        0.37772055,  0.48256479,  0.29353499,  0.14254808, -0.03712556,
        0.11134405,  0.29331302,  1.        ,  1.        ,  1.        ,
        1.        ,  0.98337472,  0.96908585,  0.9562181 ,  0.95

### Flat Run Test

A "flat run" without any disturbances verifies that the system stays at equilibrium. If variables drift during a flat run, there may be inconsistencies in the initial conditions or model parameters.

```bash
andes run case.xlsx -r tds --flat
```

## Cleanup

In [21]:
!andes misc -C
!rm -f kundur_results.csv

"/Users/hcui7/repos/andes/docs_new/source/tutorials/kundur_full_out.txt" removed.
"/Users/hcui7/repos/andes/docs_new/source/tutorials/kundur_full_out.npz" removed.
"/Users/hcui7/repos/andes/docs_new/source/tutorials/kundur_full_out.lst" removed.


## Next Steps

- {doc}`05-data-and-formats` - Loading different file formats and modifying parameters
- {doc}`06-plotting-results` - Advanced plotting and result visualization
- {doc}`07-eigenvalue-analysis` - Small-signal stability analysis